In [1]:
import numpy as np
from IPython.display import Math, display
from sympy import exp, gamma, latex, log, polygamma, simplify, symbols

In [2]:
def cprint(tex_text):
    display(Math(tex_text))

# Derivatives for the extended GAM implementation

\begin{align*}
y&: y \\
a&: \alpha = \exp(\theta_{0}) \\
n&: \eta \\
g&: \gamma \\
\end{align*}
where $\theta_{0}\in \mathbb{R}$ is an unrestricted parameter.

In [3]:
y, t0, n, g = symbols("y, theta0, eta, gamma")
a = exp(t0)

In [4]:
l_ZTNB = (
    y * log(a)
    + y * g
    - y * log(1 + a * exp(g))
    + log(gamma(y + 1 / a))
    - log(gamma(y + 1))
    - log(gamma(1 / a))
    - log(((1 + a * exp(g)) ** (1 / a)) - 1)
)

cprint(rf"\ell_{{ZTNB}} = {latex(l_ZTNB.subs(a, 'alpha'))}")

<IPython.core.display.Math object>

In [5]:
# for y == 0
l0_ZINB = -exp(n)
# for y_p = y > 0
l_ZINB = log(1 - exp(-exp(n))) + l_ZTNB

cprint(rf"\ell_{{ZINB}} = {latex(l0_ZINB)}, \quad \text{{for}} ~ y = 0.")
cprint(
    rf"\ell_{{ZINB}} = {latex(l_ZINB.subs(a, 'alpha'))}, \quad \text{{for}} ~ y > 0."
)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Derivatives, for $y > 0$

#### w.r.t. $\eta$

In [6]:
l_e = l_ZINB.diff(n, 1)
cprint(rf"\ell_{{\eta}} = {latex(l_e)}")

<IPython.core.display.Math object>

In [7]:
l_ee = l_ZINB.diff(n, 2).subs(l_e, "l_eta").expand()
cprint(rf"\ell_{{\eta\eta}} = {latex(l_ee)}")

<IPython.core.display.Math object>

In [8]:
l_eee = l_ZINB.diff(n, 3).subs(l_e, "l_eta").expand()
cprint(rf"\ell_{{\eta\eta\eta}} = {latex(l_eee)}")

<IPython.core.display.Math object>

In [9]:
l_eeee = l_ZINB.diff(n, 4).subs(l_e, "l_eta").expand()
cprint(rf"\ell_{{\eta\eta\eta\eta}} = {latex(l_eeee)}")

<IPython.core.display.Math object>

#### w.r.t. $\gamma$

In [10]:
alpha, kappa, tau = symbols("alpha kappa tau")

k = exp(g) / (1 + alpha * exp(g))
t = ((1 + alpha * exp(g)) ** (1 / alpha)) / ((1 + alpha * exp(g)) ** (1 / alpha) - 1)

In [11]:
l_g = l_ZINB.diff(g, 1).subs(a, alpha).subs(k, kappa).subs(t, tau)
cprint(rf"\ell_{{\gamma}} = {latex(l_g)}")

<IPython.core.display.Math object>

In [12]:
l_gg = l_ZINB.diff(g, 2).subs(a, alpha).subs(k, kappa).subs(t, tau).expand()

lg = symbols(r"\ell_{\gamma}")
sl_gg = -alpha * kappa * lg - kappa * tau + kappa**2 * tau**2 - kappa**2 * tau
assert simplify(sl_gg.subs(lg, l_g) - l_gg) == 0
cprint(rf"\ell_{{\gamma\gamma}} = {latex(sl_gg)}")

<IPython.core.display.Math object>

In [13]:
l_ggg = l_ZINB.diff(g, 3).subs(a, alpha).subs(k, kappa).subs(t, tau).expand()

lgg = symbols(r"\ell_{\gamma\gamma}")
sl_ggg = (
    -2 * alpha * kappa * lgg
    + lgg
    - alpha * kappa**3 * tau**2
    + alpha * kappa**3 * tau
    + 2 * kappa**2 * tau**2
    - 2 * kappa**2 * tau
    - 2 * kappa**3 * tau**3
    + 3 * kappa**3 * tau**2
    - kappa**3 * tau
)
assert simplify(sl_ggg.subs(lgg, l_gg) - l_ggg) == 0
cprint(rf"\ell_{{\gamma\gamma\gamma}} = {latex(sl_ggg)}")

<IPython.core.display.Math object>

In [14]:
l_gggg = l_ZINB.diff(g, 4).subs(a, alpha).subs(k, kappa).subs(t, tau).expand()
cprint(rf"\ell_{{\gamma\gamma\gamma\gamma}} = {latex(l_gggg)}")

<IPython.core.display.Math object>

#### w.r.t. $\theta_0$

In [15]:
omega = symbols("omega")
w = (1 / alpha) * log(1 + alpha * exp(g)) - kappa

In [16]:
l_t0 = l_ZINB.diff(t0, 1).subs(a, alpha).subs(k, kappa).subs(t, tau).subs(w, omega)
cprint(rf"\ell_{{\theta_{0}}} = {latex(l_t0)}")

<IPython.core.display.Math object>

In [17]:
l_t0t0 = l_ZINB.diff(t0, 2).subs(a, alpha).subs(k, kappa).subs(t, tau).subs(w, omega)

sl_t0t0 = (
    -alpha * kappa * lg
    + omega**2 * tau**2
    - omega**2 * tau
    - omega * tau
    - polygamma(0, 1 / alpha) / alpha
    + polygamma(0, y + 1 / alpha) / alpha
    - polygamma(1, 1 / alpha) / alpha**2
    + polygamma(1, y + 1 / alpha) / alpha**2
)
assert simplify(sl_t0t0.subs(lg, l_g) - l_t0t0) == 0
cprint(rf"\ell_{{\theta_{0}\theta_{0}}} = {latex(sl_t0t0)}")

<IPython.core.display.Math object>

#### Mixed derivatives

Mixed derivatives involving $\eta$ are all 0 in the GAMLSS case. In the Extended GAM case, with $\eta=\theta_{1} + \exp(\theta_{2})\gamma$, we are interested in mixed derivatives involving $\gamma$ and $\theta_{i}, i=0,1,2$.

In [18]:
# for quasi-Newton
l_gt0 = (
    l_ZINB.diff(g, 1, t0, 1)
    .subs(a, alpha)
    .subs(k, kappa)
    .subs(t, tau)
    .subs(w, omega)
    .expand()
)

sl_gt0 = -alpha * kappa * lg - kappa * tau**2 * omega + kappa * tau * omega
assert simplify(sl_gt0.subs(lg, l_g) - l_gt0) == 0
cprint(rf"\ell_{{\gamma\theta_{0}}} = {latex(sl_gt0)}")

<IPython.core.display.Math object>

In [19]:
# for quasi-Newton
l_ggt0 = (
    l_ZINB.diff(g, 2, t0, 1)
    .subs(a, alpha)
    .subs(k, kappa)
    .subs(t, tau)
    .subs(w, omega)
    .expand()
)

lgt0 = symbols(r"\ell_{\gamma\theta_{0}}")
sl_ggt0 = (
    -2 * alpha * kappa * lgt0
    + lgt0
    + alpha * kappa**2 * tau
    - alpha * kappa**2 * omega * tau**2
    + alpha * kappa**2 * omega * tau
    - 2 * alpha * kappa**3 * tau**2
    + 2 * alpha * kappa**3 * tau
    + 2 * kappa**2 * omega * tau**3
    - 3 * kappa**2 * omega * tau**2
    + kappa**2 * omega * tau
)
assert simplify(sl_ggt0.subs(lgt0, l_gt0) - l_ggt0) == 0
cprint(rf"\ell_{{\gamma\gamma\theta_{0}}} = {latex(sl_ggt0)}")

<IPython.core.display.Math object>

In [20]:
# for full Newton
# not simplified
l_gt0t0 = (
    l_ZINB.diff(g, 1, t0, 2)
    .subs(a, alpha)
    .subs(k, kappa)
    .subs(t, tau)
    .subs(w, omega)
    .expand()
)
cprint(rf"\ell_{{\gamma\theta_{0}\theta_{0}}} = {latex(l_gt0t0)}")

<IPython.core.display.Math object>

In [21]:
# for full Newton
# not simplified
l_ggt0t0 = (
    l_ZINB.diff(g, 2, t0, 2)
    .subs(a, alpha)
    .subs(k, kappa)
    .subs(t, tau)
    .collect(4)
    .subs(w, omega)
    .expand()
)
cprint(rf"\ell_{{\gamma\gamma\theta_{0}\theta_{0}}} = {latex(l_ggt0t0)}")

<IPython.core.display.Math object>

In [22]:
# for full Newton
# not simplified
l_gggt0 = (
    l_ZINB.diff(g, 3, t0, 1)
    .subs(a, alpha)
    .subs(k, kappa)
    .subs(t, tau)
    .collect([2, 6, 7, 12])
    .subs(w, omega)
    .expand()
)
cprint(rf"\ell_{{\gamma\gamma\gamma\theta_{0}}} = {latex(l_gggt0)}")

<IPython.core.display.Math object>

Further mixed derivatives are trivial.